<a href="https://colab.research.google.com/github/cda79/IAT360-LLM/blob/main/IAT360_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Required libraries

In [ ]:
!pip install transformers torch datasets

In [ ]:
!pip install transformers torch accelerate

##Data Pre-processing
Convert CSV to huggingface & gpt-2 format

**Import raw .csv data**

In [18]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv("shakespeare_dataset.csv")

# display it just to test
print(df.head())
df.info()

                                                                             shakespeare  \
0                                             To be, or not to be: that is the question.   
1                                                Shall I compare thee to a summer's day?   
2                                                             Wherefore art thou, Romeo?   
3                                                 If music be the food of love, play on.   
4  Some are born great, some achieve greatness, and some have greatness thrust upon 'em.   

                                                                                     modern  
0                                               Should I live or die — that's the question.  
1                                                     Should I compare you to a summer day?  
2                                                                        Why are you Romeo?  
3                                                        If music feeds

**Format it for gpt-2**

SOURCE: [Source Text] TARGET: [Target Text] <|endoftext|>

The <|endoftext|> token is for GPT-2 to signal the end of a generated sequence.


In [ ]:
def format_translation_data(row):
  # whitespace cleanup
  shakespeare_text = row['shakespeare'].strip()
  modern_text = row['modern'].strip()
  formatted_text = f"SOURCE: {shakespeare_text} TARGET: {modern_text} <|endoftext|>"
  return formatted_text

# create new column for collated data to be tokenized
df['text_for_tokenization'] = df.apply(format_translation_data, axis=1)
# display it & check
print(df[['text_for_tokenization']].head())


Full display of the text for double checking since the previous output cuts it out

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

print(df[['text_for_tokenization']].to_string())

**Format for the Huggingface trainer**

In [ ]:
hf_dataset = Dataset.from_pandas(df)
print("\nConverted to Hugging Face Dataset format.")
print(hf_dataset)

## Tokenization

**Define the tokenizer & apply**

Default gpt-2 padding & parameters.

To understand how gpt-2 tokenizes the text, [see here.](https://blog.lukesalamone.com/posts/gpt2-tokenization/)

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling

# gpt-2 tokenizer to make it machine readable
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# default padding token
tokenizer.pad_token = tokenizer.eos_token

# use the huggingface dataset we just converted
# hf_dataset = Dataset.from_pandas(df) << this was done in the previous step

def tokenize_function(examples):
    # uses the prepared 'text_for_tokenization' column
    return tokenizer(
        examples["text_for_tokenization"],
        truncation=True,
        max_length=256,
        padding="max_length"
    )

# apply tokenization
# done by mapping it to the dataset and then removing the columns without tokens
tokenized_datasets = hf_dataset.map(tokenize_function, batched=True, remove_columns=['shakespeare', 'modern', 'text_for_tokenization'])

# Display example tokenized output with the first entry
print("\n dataset structure:")
print(tokenized_datasets)
# where the input is decomposed into a string of numbers or tokens - depending on gpt-2s vocabulary
print(tokenized_datasets[0]["input_ids"])



## Training

In [ ]:
train_dataset = tokenized_datasets

# since we only have a test csv i just made it train on all the data
# otherwise we'd split it like this

#tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.1)
# or for 80/20 split, where seed is the randomizer
# ... = tokenized_datasets.train_test_split(train_size=0.8, seed=40)

**Load the model and adjust the parameters**

Loading and deploying the gpt-2 model based off tokenized data: https://medium.com/@majd.farah08/generating-text-with-gpt2-in-under-10-lines-of-code-5725a38ea685

Documentation for data collating: https://huggingface.co/docs/transformers/main/main_classes/data_collator

... and what it does internally / the different types: https://towardsdatascience.com/data-collators-in-huggingface-a0c76db798d2/

In [ ]:
from transformers import TrainingArguments, Trainer
import torch

# load gpt-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# data collating allows us to use our csv datatables as the input for model
# the mlm specifies whether we should mask tokens or not - usually used for prediction
# when its set to false then the labels = inputs with the padding ignored
# so the whole thing is read AND it returns the labels which is important for our specific translation use
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# set up the trainer and its arguements
training_args = TrainingArguments(
    output_dir="./gpt2_shakespeare_translator",  # directory to save checkpoints
    num_train_epochs=50,                       # epoch size
    per_device_train_batch_size=4,             # batch size - the default is 8 batch size which then goes to 4
    logging_steps=10,                          # log the stats of the training every 10 steps, creates a consistent loss chart
)

# initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

In [21]:
# training command
trainer.train()

Step,Training Loss
10,0.072700
20,0.096500
30,0.075500
40,0.088800
50,0.126100
60,0.127800
70,0.102800
80,0.087800
90,0.101100
100,0.077300


TrainOutput(global_step=100, training_loss=0.09563207447528839, metrics={'train_runtime': 37.499, 'train_samples_per_second': 6.667, 'train_steps_per_second': 2.667, 'total_flos': 32661504000000.0, 'train_loss': 0.09563207447528839, 'epoch': 50.0})

**Save newly trained model and reload it to replace the old one**

In [22]:
from transformers import GPT2LMHeadModel

# save model
trainer.save_model()
print(f"Model saved to {training_args.output_dir}")

#load it
model = GPT2LMHeadModel.from_pretrained(training_args.output_dir)
print("Trained model reloaded successfully.")

Model saved to ./gpt2_shakespeare_translator
Trained model reloaded successfully.


**Generate loss training chart for better visualization**

In [ ]:
import matplotlib.pyplot as plt

if 'trainer' in locals() or 'trainer' in globals():
    log_history = trainer.state.log_history

    steps = []
    losses = []

    for entry in log_history:
        if 'loss' in entry and 'step' in entry: # Filter for step-wise logs
            steps.append(entry['step'])
            losses.append(entry['loss'])

    if steps and losses:
        plt.figure(figsize=(10, 6))
        plt.plot(steps, losses, marker='o', linestyle='-')
        plt.title('Training Loss Over Steps')
        plt.xlabel('Global Step')
        plt.ylabel('Loss')
        plt.grid(True)
        plt.show()
    else:
        # If no step-wise loss was found, check for the final aggregated training_loss
        final_train_loss_entry = None
        for entry in log_history:
            if 'train_loss' in entry:
                final_train_loss_entry = entry
                break

        if final_train_loss_entry:
            print(f"Only the final training loss is available: {final_train_loss_entry['train_loss']} at step {final_train_loss_entry['step']}.")
            print("A graph requires multiple data points. To see loss progression, ensure 'logging_steps' is set appropriately in TrainingArguments and logs are accessible.")
            # Optionally, plot a single point if that's all there is
            plt.figure(figsize=(2, 2))
            plt.plot([final_train_loss_entry['step']], [final_train_loss_entry['train_loss']], marker='X', color='red', markersize=10)
            plt.title('Final Training Loss')
            plt.xlabel('Global Step')
            plt.ylabel('Loss')
            plt.grid(True)
            plt.show()
        else:
            print("No training loss data found.")
else:
    print("The 'trainer' object was not found.")

## Deploy Model & Generate Text

Test model by using custom inputs and reviewing the outputs based on training.

###**Important**
the input sentence must end w/ "TARGET:" so the model can generate its response afterwards. This is how it tries to complete the entire sentence based off the patterns learned during training.


*   Where we formatted and collated the sentences to start with `SOURCE` and `TARGET` from before:
*   `formatted_text = f"SOURCE: {shakespeare_text} TARGET: {modern_text} <|endoftext|>"`




Generation part of the code and its parameters were based off the huggingface text documentation: https://huggingface.co/docs/transformers/en/main_classes/text_generation

as well as gemini's suggestions for what should be fine tuned

In [38]:
from transformers import pipeline

# huggingface pipeline library
generator = pipeline(
    'text-generation', # task
    model=model, # saved model
    tokenizer=tokenizer, # use previous established tokenizer
    device=0 if torch.cuda.is_available() else -1 # Use GPU if available
)

## INPUT PART !!!
input_shakespeare = "Goodbye, my lord." #exact test from the training set to check it works, replace this later
prompt = f"SOURCE: {input_shakespeare} TARGET:" # actual formatted prompt the generator will be recieving

## generator parameters
output = generator(
    prompt, #prompt as input
    max_new_tokens=30, #output length
    num_return_sequences=1, # "The number of independently computed returned sequences for each element in the batch."
    do_sample=True, #generation strategy
    top_k=50, #probablity tokens, use 50 first most likely
    temperature=0.7, #randomness
    eos_token_id=tokenizer.eos_token_id # stop generation when the <|endoftext|> token is generated
)

## generated text extraction + stripping
generated_text = output[0]['generated_text']
# remove the PROMPT: text from the generated line and the end <|endoftext|> token
translation = generated_text.replace(prompt, '').replace(tokenizer.eos_token, '').strip()

## FINAL OUTPUT
print(f"\n--- Translation Output ---")
print(f"Input Prompt: {input_shakespeare}")
print(f"Generated Modern English: {translation}")




Device set to use cuda:0



--- Translation Output ---
Input Prompt: Goodbye, my lord.
Generated Modern English: Should I live or die — that's the question.  If you live or die, play on.  I won't tell you to
